In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

# 1. Veri Oluşturma veya Yükleme
# Örnek veri oluşturma (CSV'den de okunabilir)

In [4]:
data = {
    'Ad': ['Ahmet', 'Mehmet', 'Ayşe', 'Fatma', 'Ali', 'Ayşe', 'Ahmet', 'Fatma', 'Mehmet', 'Ali'],
    'Yaş': [25, 30, 22, 35, 28, 22, np.nan, 35, 30, np.nan],
    'Maaş': [5000, 6000, 5200, 7000, 6800, 5200, 5000, 7000, 6000, 6800],
    'Şehir': ['İstanbul', 'Ankara', 'İzmir', 'Bursa', 'Antalya', np.nan, 'İstanbul', 'Bursa', 'Ankara', 'Antalya'],
    'Çalışma Süresi (Yıl)': [2, 5, 1, 10, 7, 1, 2, 10, 5, 7],
    'Departman': ['IT', 'HR', 'IT', 'Finans', 'IT', 'HR', 'IT', 'Finans', 'HR', 'IT']
}

df = pd.DataFrame(data)

# Gerekirse CSV'den okuma:
# df = pd.read_csv('veri.csv')

# 2. Veriyi İnceleme

In [5]:
print("Orijinal Veri:\n", df)
print("\nVeri Tipleri:\n", df.dtypes)
print("\nEksik Veri Sayısı:\n", df.isnull().sum())


Orijinal Veri:
        Ad   Yaş  Maaş     Şehir  Çalışma Süresi (Yıl) Departman
0   Ahmet  25.0  5000  İstanbul                     2        IT
1  Mehmet  30.0  6000    Ankara                     5        HR
2    Ayşe  22.0  5200     İzmir                     1        IT
3   Fatma  35.0  7000     Bursa                    10    Finans
4     Ali  28.0  6800   Antalya                     7        IT
5    Ayşe  22.0  5200       NaN                     1        HR
6   Ahmet   NaN  5000  İstanbul                     2        IT
7   Fatma  35.0  7000     Bursa                    10    Finans
8  Mehmet  30.0  6000    Ankara                     5        HR
9     Ali   NaN  6800   Antalya                     7        IT

Veri Tipleri:
 Ad                       object
Yaş                     float64
Maaş                      int64
Şehir                    object
Çalışma Süresi (Yıl)      int64
Departman                object
dtype: object

Eksik Veri Sayısı:
 Ad                      0
Yaş        

# 3. Eksik Veri Yönetimi
# Sayısal sütunları ortalama ile, kategorik sütunları 'Bilinmiyor' ile doldur

In [6]:
for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        df[col] = df[col].fillna(df[col].mean())
    else:
        df[col] = df[col].fillna('Bilinmiyor')

# 4. Tekrarlayan Verileri Kaldırma

In [7]:
df = df.drop_duplicates()

# 5. Kategorik Verileri Kodlama (Encoding)
# Label Encoding

In [8]:
label_encoder = LabelEncoder()
for col in ['Şehir', 'Departman']:
    df[col] = label_encoder.fit_transform(df[col])

# One-Hot Encoding (Alternatif)
# df = pd.get_dummies(df, columns=['Şehir', 'Departman'], drop_first=True)

# 6. Sayısal Verileri Ölçeklendirme (Scaling)
# MinMaxScaler

In [9]:
scaler = MinMaxScaler()
numerical_cols = ['Yaş', 'Maaş', 'Çalışma Süresi (Yıl)']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# 7. Özellik Seçimi (Feature Selection)
# Hedef değişkeni ve özellikleri ayır

In [10]:
X = df.drop(['Ad'], axis=1)  # 'Ad'ı çıkarıyoruz, çünkü model için gerekli değil
y = df['Ad']  # Hedef değişken olarak 'Ad'ı kullanıyoruz

In [11]:
# En iyi 3 özelliği seç
selector = SelectKBest(score_func=f_classif, k=3)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support(indices=True)]
print("\nSeçilen Özellikler:", selected_features)


Seçilen Özellikler: Index(['Yaş', 'Maaş', 'Çalışma Süresi (Yıl)'], dtype='object')


/opt/miniconda3/envs/data-analysis/lib/python3.13/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


# 8. Eğitim ve Test Verisi Oluşturma

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X[selected_features], y, test_size=0.2, random_state=42
)

print("\nEğitim Verisi Boyutu:", X_train.shape)
print("Test Verisi Boyutu:", X_test.shape)

print("\nİşlenmiş Veri:\n", df)


Eğitim Verisi Boyutu: (6, 3)
Test Verisi Boyutu: (2, 3)

İşlenmiş Veri:
        Ad       Yaş  Maaş  Şehir  Çalışma Süresi (Yıl)  Departman
0   Ahmet  0.230769   0.0      4              0.111111          2
1  Mehmet  0.615385   0.5      0              0.444444          1
2    Ayşe  0.000000   0.1      5              0.000000          2
3   Fatma  1.000000   1.0      3              1.000000          0
4     Ali  0.461538   0.9      1              0.666667          2
5    Ayşe  0.000000   0.1      2              0.000000          1
6   Ahmet  0.490385   0.0      4              0.111111          2
9     Ali  0.490385   0.9      1              0.666667          2
